In [44]:
import numpy as np
import math

# B 4 本読み第 5 回 
## 内積と直交性
2020/05/08  
実装担当：松林慶祐 / テスト担当：重見開


## 目次

- 実装環境
- ユークリッド空間
- n- ノルム
- コサイン類似度
- 正規直交基底とユニタリ行列
- グラム・シュミットの正規直交化
- 正射影
- まとめ


## 環境

- 実装環境
    - macOS Catalina (10.15.4)
    - Python 3.7.7
- スライド制作環境
    - Windows Home 64bit (1903)
    - Jupyter Notebook 6.0.3
    - Python 3.7.7


## ユークリッド空間<br>
ユークリッド内積を導入した空間のこと．
一般的に知られている内積はユークリッド内積といい，<br>
二つのベクトル $\boldsymbol{x},\boldsymbol{y}\in{\mathbb{C}^{N}}$ に対して，以下の式で表される.  
*$ \langle \boldsymbol{x},\boldsymbol{y} \rangle = \boldsymbol{y}^H
\boldsymbol{x} = x_1\overline{y_1}+x_2\overline{y_2}+\cdots+x_N\overline{y_N} $*
<br><br>
また，内積はスカラであることから，順番を入れ替えると共役になる．  
$ \langle \boldsymbol{y},\boldsymbol{x} \rangle =\boldsymbol{x}^H\boldsymbol{y}= (\boldsymbol{y}^H\boldsymbol{x})^H = \overline{\boldsymbol{y}^H\boldsymbol{x}}=\overline{\langle \boldsymbol{x},\boldsymbol{y} \rangle}$  
  
実数の N 次元ユークリッド空間の場合は，  
共役を考える必要がないため，簡潔に表記できる．  
$ \langle \boldsymbol{x},\boldsymbol{y} \rangle = \boldsymbol{y}^T
\boldsymbol{x} = x_1{y_1}+x_2{y_2}+\cdots+x_N{y_N} $

## ベクトルの直交性  
二つのベクトル $\boldsymbol{x},\boldsymbol{y}\in{\mathbb{C}^{N}}$ の内積に対して， 
$$\langle \boldsymbol{x},\boldsymbol{y} \rangle=0$$ <br>
が成り立つとき，<br>
$\boldsymbol{x}$ と $\boldsymbol{y}$ は<strong>直交する</strong>という．

## 実装課題 1<br>
ユーグリッド内積を求める（ついでに直交性も判定）関数を実装

## 関数の動作<br>

1. 二つのベクトルを入力（縦ベクトルを想定）
1. 縦と横の場合はエラー，横同士の場合は縦に変換
1. 2つ目に入力されたベクトルを共役なものに変換
1. 成分同士を掛け合わせて加算
1. 内積の結果が 0 であったら直交であると判断
1. 結果を縦ベクトルで出力


行列ベクトルを表すためにnumpyを使用

In [45]:
import numpy as np

In [46]:
class InnerProduct:  # 内積値と直交性かどうかのステートを持つクラス

    def __init__(self, result, orthogonal):
        self.__ip = result
        self.__orth = orthogonal

    @property
    def ip(self):
        return self.__ip

    @property
    def orth(self):
        return self.__orth


In [47]:
def euclidean_inner_product(vector1, vector2):
    # numpy配列変換
    x = np.array(vector1)
    y = np.array(vector2)

    # エラー処理
    if not x.shape == y.shape:
        raise ValueError("ベクトルのサイズが違います")
    else:
        pass

    # 共役をとる
    conj_y = y.conj()

    # 成分ごとの積をとって加算
    tmp = []
    for i in range(x.size):
        tmp.append(x[i][0] * conj_y[i][0])
    res = sum(tmp)

    # 直交性の判断
    if round(res, 7) == 0:
        is_orthogonal = True
    else:
        is_orthogonal = False

    return InnerProduct(res, is_orthogonal)

In [48]:
x = [[2-1j], [1j], [-1j]]
y = [[3j], [1+1j], [4+1j]]
x_in_y = euclidean_inner_product(x,y)
print("<x,y> = {0}，直交かどうか={1}".format(x_in_y.ip, x_in_y.orth))
a = [[2], [3], [-1]]
b = [[4], [-2], [2]]
a_in_b = euclidean_inner_product(a, b)
print("<a,b> = {0}，直交かどうか={1}".format(a_in_b.ip, a_in_b.orth))

<x,y> = (-3-9j)，直交かどうか=False
<a,b> = 0，直交かどうか=True


## ノルム<br>
ベクトルの長さを決める量のこと．<br>
内積空間のベクトル $\boldsymbol{x}$ に対して<br>
$$\|\boldsymbol{x}\| = \sqrt{\langle\boldsymbol{x},\boldsymbol{x} \rangle}$$<br>
をノルムと呼ぶ．<br><br>
特に，ノルムが 1 のとき，$\boldsymbol{x}$ は<strong>単位ベクトル</strong>であるという．

ユークリッド空間のベクトル $\boldsymbol{x}\in{\mathbb{C}^{N}}$ の場合に，ノルムは<br>
$$\|\boldsymbol{x}\| = \sqrt{\sum_{i=1}^{N}|x_i|^2}$$<br>
のように要素で書き下すことができ，特にこのノルムを $l_2$ ノルム，または 2- ノルムと呼ぶ．<br><br>
一方，<br>
$$\|\boldsymbol{x}\|_1 = \sum_{i=1}^{N}|x_i|$$<br>
のように定義されるノルムを，$l_1$ ノルム，または 1- ノルムとよび，信号処理や機械学習で疎性（スパース性）を測るのに広く使われる．

## 実装課題 2<br>
n- ノルムを求める（引数はベクトルとn）関数を実装

## 関数の動作<br>

1. ノルムを求めるベクトルと n を入力する
2. ベクトルの成分ごとに絶対値をとり，n 乗する
1. n 乗したものの総和をとり，n 乗根を求める


In [49]:
def n_norm(vector, n=2):
    # 縦ベクトルに変換
    vec = np.array(vector).reshape(-1, 1)

    # 計算
    tmp = []
    for i in range(vec.size):
        # print(abs(vec[i][0]))
        tmp.append(abs(vec[i][0]) ** n)  # 絶対値のn乗(たぶん複素数もOK)

    # 全部加算してn乗根
    res = sum(tmp)**(1 / n)

    return res


In [50]:
xx = [[-3], [2], [2], [3]]
print("整数のみ {0}".format(xx))
xxnorm1 = n_norm(xx,1)
xxnorm2 = n_norm(xx)
print("1- ノルムは {0}\n2- ノルムは {1}".format(xxnorm1,xxnorm2))

yy = [[-3], [5.6], [4+1j]]
print("異なる型同士 {0}".format(yy))
yynorm1 = n_norm(yy,1)
yynorm2 = n_norm(yy)
print("1- ノルムは {0}\n2- ノルムは {1}".format(yynorm1,yynorm2))

整数のみ [[-3], [2], [2], [3]]
1- ノルムは 10.0
2- ノルムは 5.0990195135927845
異なる型同士 [[-3], [5.6], [(4+1j)]]
1- ノルムは 12.72310562561766
2- ノルムは 7.573638491504595


## コサイン類似度<br>
ノルムと内積によって導入される，二つのベクトルの角度のことで，パターン間の距離を測るのに用いられる．
$$\cos{\theta}=\frac{\langle\boldsymbol{x},\boldsymbol{y} \rangle}{\|\boldsymbol{x}\|\|\boldsymbol{y}\|}$$<br><br>
ベクトル同士の向きが近いほどなす角度が小さくなるので，コサイン類似度は小さくなり，向きが異なるほど，コサイン類似度は大きくなる．<br><br>
コサイン類似度は -1 から 1 の値をとり，<br>コサイン類似度から $0\leq\theta\leq\pi$ の範囲で決まる $\theta$ を<strong>角度</strong>と呼ぶ．

## 実装課題 3<br>
コサイン類似度を求める関数を実装

## 関数の動作<br>

1. 二つのベクトルを入力
1. 課題 1 の関数を使って内積を計算
1. 課題 2 の関数を使ってそれぞれのベクトルの 2- ノルムを計算
1. 定義に従ってコサイン類似度を計算


In [51]:
def cosine_similarity(vector1, vector2):
    # 内積計算
    ip_xy = euclidean_inner_product(vector1, vector2).ip

    # ノルム計算
    norm_x = n_norm(vector1)
    norm_y = n_norm(vector2)

    if norm_x == 0 or norm_y == 0:
        raise ZeroDivisionError("ノルムが0です")
    else:
        pass

    # コサイン類似度計算
    res = ip_xy / (norm_x * norm_y)

    return res

In [52]:
xxx = [[0.4], [5.6], [-0.07]]
yyy = [[-1.1], [0.22], [-3.3]]
print("コサイン類似度は {0}".format(cosine_similarity(xxx,yyy)))

コサイン類似度は 0.05227442125730368


## 正規直交基底<br>
N次元ベクトル空間の基底ベクトル $\{\boldsymbol{u}_i\}_{i=1}^N$ が<br>

- すべて互いに直交
- すべての基底ベクトルのノルムが 1  

のとき，この基底を<strong>正規直交基底</strong>と呼ぶ.<br><br>


## 正規直交展開<br>
N次元ベクトル空間 $V$ の基底 $\{\boldsymbol{u}_i\}_{i=1}^N$ が正規直交性を満たすとき，$V$ 内のベクトル $\boldsymbol{x}\in{V}$ は基底 $\{\boldsymbol{u}_i\}_{i=1}^N$ を用いて，
$$\boldsymbol{x}=\langle\boldsymbol{x},\boldsymbol{u}_1\rangle\boldsymbol{u}_1+\langle\boldsymbol{x},\boldsymbol{u}_2\rangle\boldsymbol{u}_2+\cdots+\langle\boldsymbol{x},\boldsymbol{u}_N\rangle\boldsymbol{u}_N$$
と展開できる．これを<strong>正規直交展開</strong>という．<br>
$\langle\boldsymbol{x},\boldsymbol{u}_N\rangle$ が展開係数となっている．

## ユニタリ行列<br>
ある行列の逆行列とエルミート転置が一致しているような行列のこと．<br>
すなわち，$\boldsymbol{A}^{-1}=\boldsymbol{A}^H$ より$\boldsymbol{A}\boldsymbol{A}^H=\boldsymbol{A}^H\boldsymbol{A}=\boldsymbol{I}$<br>
が成り立つような行列のことをユニタリ行列という．<br><br>
ユニタリ行列の列ベクトルは正規直交性を持つ．

正規直交展開の展開係数 $c_i =\langle\boldsymbol{x},\boldsymbol{u}_i\rangle=\boldsymbol{u}_i^H\boldsymbol{x}$ を並べたベクトルを考えると，
$$\boldsymbol{c}=\left[\begin{array}{c}\boldsymbol{u}_1^H\boldsymbol{x}\\\boldsymbol{u_2}^H\boldsymbol{x} \\ \vdots \\ \boldsymbol{u}_N^H\boldsymbol{x}\end{array} \right]=[\boldsymbol{u}_1,\boldsymbol{u}_2,\cdots,\boldsymbol{u}_N]^H\boldsymbol{x} = \boldsymbol{U}^H\boldsymbol{x}$$
のように表せる．この時 $\boldsymbol{U}$ は正規直交基底が列ベクトルとなっている行列である．<br>
つまり $\boldsymbol{U}$ の列ベクトルが正規直交性をもつので，$\boldsymbol{U}$ はユニタリ行列である．<br><br>
ノルムについて考えると，<br>
$\|\boldsymbol{c}\|^2= \|\boldsymbol{U^Hx}\|^2=\|\boldsymbol{x}\|^2$<br>
となっていることから，ユニタリ行列は<strong>ノルム変えない</strong>性質を持つ．


## 実装課題 4<br>
ユニタリ行列かどうか判定する関数を実装

判別方法

1. $\boldsymbol{A}^{-1}=\boldsymbol{A}^H$ （逆行列とエルミート転置が一致する）
1. $\boldsymbol{A}\boldsymbol{A}^H=\boldsymbol{A}^H\boldsymbol{A}=\boldsymbol{I}$ （逆行列とエルミート転置の積が単位行列になる）



&rArr; 2番の方法を採用

## 関数の動作<br>

1. 行列を入力
1. 入力された行列のエルミート転置を取得
1. 行列とそのエルミート転置の積を計算
1. 用意しておいた単位行列と積が等しいか判別


In [53]:
def unitary_matrix(matrix):
    
    uni = np.array(matrix)
    her_mat = np.conjugate(uni.T)
    ide_mat = np.dot(uni, her_mat)
    ide_mat2 = np.identity(len(uni))

    for i in range(len(uni)):
        for j in range(len(uni)):
            if not round(ide_mat[i][j] - ide_mat2[i][j], 7) == 0:
                unitary = False
            else:
                unitary = True
    return unitary

In [54]:
mat = (1/math.sqrt(2)) * np.array([[1,1],[-1j,1j]])
print(unitary_matrix(mat))
mat2 = np.array([[1,1],[-1j,1j]])
print(unitary_matrix(mat2))

True
False


## グラム・シュミットの正規直交化<br>
直交ではない基底から，正規直交基底を生成する方法のこと．<br>
以下の手順により，直交しているとは限らない基底 $\{\boldsymbol{u}_i\}_{i=1}^N$ から 正規直交基底 $\{\boldsymbol{v}_i\}_{i=1}^N$ を得ることができる．

<ol style="list-style-type: upper-latin">
    <li>$\begin{align*}&\boldsymbol{v_1}=\frac{\boldsymbol{u_1}}{\|u_1\|}\end{align*}$</li>
    <li>$i=2,3,\cdots,N$に対し以下の操作を繰り返す
        <ol style="list-style-type: lower-latin">
    <li>$\begin{align*}&\hat{\boldsymbol{u_i}}= \sum_{j=1}^{i-1}\langle\boldsymbol{u_i},\boldsymbol{v_j}\rangle\boldsymbol{v_j}\end{align*}$ </li>
    <li>$\begin{align*}&\boldsymbol{v_i} = \frac{\boldsymbol{u_i}-\hat{\boldsymbol{u_i}}}{\|\boldsymbol{u_i}-\hat{\boldsymbol{u_i}}\|}\end{align*}$</li></ol></li>
</ol>

## 実装課題 5<br>
基底をグラム・シュミットの正規直交化する関数を実装

## 関数の動作

1. 正規直交化したい基底を並べた行列を入力
1. 直交化の手順 A のとおりに 1 本目の基底を計算
1. $i=2,3,\cdots,N$に対し for 文を回すことで直交化手順 B の a と b を計算し残りの基底を計算


In [55]:
def Gram_Schmidt_orthonormalization(matrix):
    # numpy基底行列matrix_u
    matrix_u = np.array(matrix)
    u_len = len(matrix_u)

    for i in range(u_len):
        is_norm_zero = n_norm(matrix_u[:, i])
        if is_norm_zero == 0:
            raise ZeroDivisionError("ノルムが0です")
        else:
            pass

    # 第一手順
    u0 = np.array(matrix_u[:, 0]).reshape(-1, 1)  # 一つ目の基底を縦ベクトルで取得
    v0 = u0/(n_norm(u0))
    matrix_v = np.array(v0).reshape(-1, 1)

    # 第二手順
    for i in range(1, u_len):
        ui = np.array(matrix_u[:, i]).reshape(-1, 1)
        u_ = np.zeros((u_len, 1))
        for j in range(i):
            vj = np.array(matrix_v[:, j]).reshape(-1, 1)
            u_ = u_ + euclidean_inner_product(ui, vj).ip * vj
        vi = (ui - u_) / (n_norm((ui - u_)))
        matrix_v = np.hstack((matrix_v, vi))
    return matrix_v

In [56]:
u = np.array([[1,1,1],[1,-1,2],[-1,1,3]]).T
print(Gram_Schmidt_orthonormalization(u))


[[ 0.57735027  0.15430335 -0.80178373]
 [ 0.57735027 -0.77151675  0.26726124]
 [ 0.57735027  0.6172134   0.53452248]]


## 正射影<br>
N次元空間 $V$ に，ある部分空間 $W$ を定義したとき，任意の $\boldsymbol{x}\in{V}$ に対して<br>
$\begin{align*}&\langle\boldsymbol{y},\boldsymbol{x-y}\rangle=0\end{align*}$<br>
となるような $\boldsymbol{y}\in{W}$ を，$x$の$W$への<strong>正射影（直交射影）</strong>という．

$V$ の基底 $\{\boldsymbol{u}_i\}_{i=1}^N$ が正規直交性を満たすとき，部分空間 $W$ に対する $\boldsymbol{x}$ の正射影を特に $\boldsymbol{P}_w\boldsymbol{x}$ と表記し，以下のように表すことができる．<br>
$$\boldsymbol{P}_w\boldsymbol{x}=\sum_{i=1}^{r}\langle\boldsymbol{x},\boldsymbol{u_i}\rangle\boldsymbol{u_i}=\left(\sum_{i=1}^{r}\boldsymbol{u_i}\boldsymbol{u_i}^H \right)\boldsymbol{x}$$

## 実装課題 6<br>
正射影を求める関数を実装

## 関数の動作<br>

1. 基底を並べた行列と射影したいベクトルを入力する．
1. 1つ目の基底ベクトルと射影したいベクトルの内積を計算し，1つ目の基底ベクトルをその値でスカラ倍する．
1. 2つ目の基底ベクトルについても同様に行い，1つ目の結果とのベクトル和をとる．
1. それ以降も同様にして，基底ベクトルを内積でスカラ倍したものの総和をとる．
1. ベクトルが総和されたものを出力する．


In [57]:
def orthogonal_projection(matrix, vector):
    x = np.array(vector).reshape(-1, 1)
    matrix_u = np.array(matrix)
    u_len = len(matrix_u)
    pwx = np.zeros((u_len, 1))

    for i in range(u_len):
        ui = np.array(matrix_u[:, i]).reshape(-1, 1)
        pwx = pwx + euclidean_inner_product(x, ui).ip * ui
    return pwx

In [58]:
x6 = [[2], [3], [-1]]
Pw = [[5/6, 1/3, 1/6], [1/3, 1/3, -1/3], [1/6, -1/3, 5/6]]
print(orthogonal_projection(Pw, x6))

[[ 2.5]
 [ 2. ]
 [-1.5]]


## まとめ<br>

- ユークリッド内積や正規直交性について理解した
- Jupyter Notebook の使い方について理解した
    - OS の差を意識することなく使えた
    - RISE でのスライドづくりは Keynote,Powerpoint に比べて手間がかかった
- 例外処理の実装が難しかった